In [81]:
import mayavi.mlab as myv
import numpy as np
import scipy as sp


In [82]:
colours = [(0,0,1), (0,1,0), (1,0,0), (0.65,0.65,0.65), (0,0.5,0.5), (0.5,0,0.5), (1,1,0), (1,1,1)]
# colours = [(0,0,1), (0,1,0), (1,0,0), (0.65,0.65,0.65), (0.65,0.65,0.65), (0.65,0.65,0.65), (0.65,0.65,0.65), (0.65,0.65,0.65)]
line_size = 0.4
point_size = 2
add_vertices = False

corner_list_origin = np.array([(0,0,0), (0,0,1), (0,1,0), (1,0,0), (0,1,1), (1,1,0), (1,0,1), (1,1,1)]) 

central_origin = np.array([(0.5, 0.5, 0.5)]) 

face_centre_list_origin = np.array([(0,0.5,0.5), (0.5,1,0.5), (1,0.5,0.5), (0.5,0,0.5), (0.5,0.5,1), (0.5,0.5,0)]) 

corner_corner_struts = ((1,0,2,4,1,6,3,5,7,6), (7,4,2,5,3,0))

corner_face_struts = (1,8,2,9,7,10,3,11,1,12,7,10,3,13,2), (0,8,4,9,5,10,6,11,0,13,5,10,6,12,4) # With face centers added

corner_central_struts = (0,8,7,4,8,3,5,8,1,6,8,2) # With corner center added

face_centre_components_list = [(0,1,2,4), (2,4,5,7), (3,5,6,7), (0,1,3,6)]


In [83]:
def add_axis():
    x_axis = myv.quiver3d(0,0,0,1,0,0,color=(1,0,0),line_width=2.0,scale_factor=100.0)
    y_axis = myv.quiver3d(0,0,0,0,1,0,color=(0,1,0),line_width=2.0,scale_factor=100.0)
    z_axis = myv.quiver3d(0,0,0,0,0,1,color=(0,0,1),line_width=2.0,scale_factor=100.0)

In [84]:
def fcc_cell(corner_list, central, face_centre_list, strut_thickness:float):
    # Cell 8 corner bounding box
    for row in range(2):
        strut = np.array([corner_list[idx] for idx in corner_corner_struts[row]])
        myv.plot3d(strut[:,0], strut[:,1], strut[:,2], color=colours[3], tube_radius=strut_thickness)   

    shrunken_corner_list = corner_list.astype(np.float64)
    shrunken_corner_list += (central[0] - corner_list) * 0.02 # Shrink corners inward a little so these struts do not overwrite the corner bounding box struts
    face_list = np.concatenate((shrunken_corner_list, face_centre_list))
    # FCC face center to corner connections
    for row in range(2):
        strut = np.array([face_list[idx] for idx in corner_face_struts[row]])
        myv.plot3d(strut[:,0], strut[:,1], strut[:,2], color=colours[4], tube_radius=strut_thickness*0.95)   

    if add_vertices:
        # FCC vertices
        myv.points3d(corner_list[:, 0], corner_list[:, 1], corner_list[:, 2], color=colours[2], scale_factor=point_size)
        myv.points3d(face_centre_list[:, 0], face_centre_list[:, 1], face_centre_list[:, 2], color=colours[1], scale_factor=point_size)


In [85]:
def bcc_cell(corner_list:np.array, central:np.array, strut_thickness:float):    
    # Cell 8 corner bounding box
    for row in range(2):
        strut = np.array([corner_list[idx] for idx in corner_corner_struts[row]])
        myv.plot3d(strut[:,0], strut[:,1], strut[:,2], color=colours[3], tube_radius=strut_thickness)   

    shrunken_corner_list = corner_list.astype(np.float64)
    shrunken_corner_list += (central[0] - corner_list) * 0.02 # Shrink corners inward a little so these struts do not overwrite the corner bounding box struts
    # print(corner_list, shrunken_corner_list)
    central_list = np.concatenate((shrunken_corner_list, central))
    # BCC Central atom connection
    strut = np.array([central_list[idx] for idx in corner_central_struts])
    myv.plot3d(strut[:,0], strut[:,1], strut[:,2], color=colours[5], tube_radius=strut_thickness*0.95)   

    if add_vertices:
        # BCC vertices
        myv.points3d(corner_list[:, 0], corner_list[:, 1], corner_list[:, 2], color=colours[2], scale_factor=point_size)
        myv.points3d(central[:, 0], central[:, 1], central[:, 2], color=colours[0], scale_factor=point_size)

In [86]:
def locate_fcc_cell(offset:np.array, cell_size:float, rotation:np.array):
    offset = np.array(offset)
    offset_a = np.repeat(offset[np.newaxis,:], corner_list_origin.shape[0], axis=0)
    offset_b = np.repeat(offset[np.newaxis,:], face_centre_list_origin.shape[0], axis=0)


    corner_list = (corner_list_origin*cell_size)@rotation + offset_a
    face_centre_list = (face_centre_list_origin*cell_size)@rotation + offset_b
    central = (central_origin*cell_size)@rotation + offset_b
    # print(corner_list_origin*cell_size, offset_a, corner_list, rotation)

    return [corner_list, central, face_centre_list]


def locate_bcc_cell(offset:np.array, cell_size:float, rotation:np.array=None):
    offset = np.array(offset)
    offset_a = np.repeat(offset[np.newaxis,:], corner_list_origin.shape[0], axis=0)
    offset_b = np.repeat(offset[np.newaxis,:], central_origin.shape[0], axis=0)
    if rotation is not None:
        corner_list = (corner_list_origin*cell_size)@rotation + offset_a
        central = (central_origin*cell_size)@rotation + offset_b
    else:
        corner_list = (corner_list_origin*cell_size) + offset_a
        central = (central_origin*cell_size) + offset_b
        
    return [corner_list, central]

In [87]:
cos = np.cos
sin = np.sin
def rotation_matrix_from_euler_angles(roll:float=0, pitch:float=0, yaw:float=0):
    roll_arr = np.array([
        [1, 0, 0],
        [0, cos(roll), -sin(roll)],
        [0, sin(roll), cos(roll)]])    

    pitch_arr = np.array([
        [cos(pitch), 0, sin(pitch)],
        [0, 1, 0],
        [-sin(pitch), 0, cos(pitch)]])
    
    yaw_arr = np.array([
        [cos(yaw), -sin(yaw), 0],
        [sin(yaw), cos(yaw), 0],
        [0, 0, 1]])
    
    return yaw_arr @ pitch_arr @ roll_arr 

In [88]:
corner_merging_list = [((4,1), (2,0), (7,6), (5,3)), ((1,0), (4,2), (7,5), (6,3)), ((3,0), (5,2), (6,1), (7,4))]
'''For (right, left), (up, down) and (front, back)'''

def merge_corners(cell_1_corners, cell_2_corners, merge_list, midpoint:bool=True):
    for pair in merge_list:
        corner_1 = cell_1_corners[pair[0]]
        corner_2 = cell_2_corners[pair[1]]
        if midpoint:
            centerpoint = (corner_1 + corner_2)/2
        else:
            centerpoint = corner_1
        cell_1_corners[pair[0]] = centerpoint
        cell_2_corners[pair[1]] = centerpoint
    return cell_1_corners, cell_2_corners



def merge_cell_corners(cell_1_corners, cell_2_corners, axis, midpoint:bool):
    if axis == "x":
        cell_1_corners, cell_2_corners = merge_corners(cell_1_corners, cell_2_corners, corner_merging_list[2], midpoint)
    if axis == "z":
        cell_1_corners, cell_2_corners = merge_corners(cell_1_corners, cell_2_corners, corner_merging_list[1], midpoint)
    else:
        cell_1_corners, cell_2_corners = merge_corners(cell_1_corners, cell_2_corners, corner_merging_list[0], midpoint)
    return cell_1_corners, cell_2_corners
    

In [89]:
def connect_corners(corner_dict_a:dict, corner_dict_b:dict, strut_thickness, threshold_dist:float, limit_single:bool=False):
    for corner_b in corner_dict_b.keys():
        min_dist = np.inf
        strut = None
        for corner_a in corner_dict_a.keys():
            # if corner_dict_a < 
            corner_a = np.array(corner_a)
            corner_b = np.array(corner_b)
            dist = np.linalg.norm(corner_a - corner_b)
            if limit_single:
                if dist < threshold_dist and dist < min_dist:
                    min_dist = dist
                    strut = np.vstack((corner_a, corner_b))

            elif dist < threshold_dist:
                strut = np.vstack((corner_a, corner_b))
                myv.plot3d(strut[:,0], strut[:,1], strut[:,2], color=colours[-2], tube_radius=strut_thickness)   

        if limit_single and strut is not None:
            myv.plot3d(strut[:,0], strut[:,1], strut[:,2], color=colours[-2], tube_radius=strut_thickness)   

In [ ]:
def create_gradient_lattice(cell_size_arr:list, z_height:float, max_rotation_per_cell:float=10):

    lattice_array = []

    y = 5
    rotation_increment = 0
    
    for cell_size in cell_size_arr:
        lattice_array.append([])

        z = 5
        for _ in range(z_height//cell_size):

            roll, pitch, yaw = np.deg2rad(np.random.random() * max_rotation_per_cell - max_rotation_per_cell/2 + rotation_increment), np.deg2rad(np.random.random() * max_rotation_per_cell - max_rotation_per_cell/2), np.deg2rad(np.random.random() * max_rotation_per_cell - max_rotation_per_cell/2)
            rotation = rotation_matrix_from_euler_angles(roll, pitch, yaw)
            # rotation = rotation_matrix_from_euler_angles(0,0,0)
            offset = np.array((5,y,z))
            
            if cell_size > 15:
                vertices = locate_bcc_cell(offset,cell_size,rotation)
            else:
                vertices = locate_fcc_cell(offset,cell_size,rotation)
            # print(vertices)
            lattice_array[-1].append(vertices)

            z += cell_size

        y += cell_size
        # z_height -= z_height%cell_size
        # rotation_increment += 3


    # Merge sideways
    for column_idx in range(len(lattice_array) - 1):
        if cell_size_arr[column_idx] > cell_size_arr[column_idx+1]:
            previous_row_idx = 0
            for row_idx in range(len(lattice_array[column_idx+1])):
                if previous_row_idx == len(lattice_array[column_idx]):
                    break
                
                centre_point = lattice_array[column_idx][previous_row_idx][1]
                if lattice_array[column_idx+1][row_idx][0][1,2] < centre_point[0,2]:
                    pass
                else:
                    corner_list_1 = lattice_array[column_idx][previous_row_idx][0]
                    corner_list_2 = lattice_array[column_idx+1][row_idx][0]
                    corner_list_1, corner_list_2 = merge_cell_corners(corner_list_1, corner_list_2, "y", True)
                    lattice_array[column_idx][previous_row_idx][0] = corner_list_1
                    lattice_array[column_idx+1][row_idx][0] = corner_list_2
                    previous_row_idx += 1

        else:
            row_idx = 0
            for previous_row_idx in range(len(lattice_array[column_idx])):
                if row_idx == len(lattice_array[column_idx+1]):
                    break

                centre_point = lattice_array[column_idx+1][row_idx][1]
                if lattice_array[column_idx][previous_row_idx][0][1,2] < centre_point[0,2]:
                    pass
                else:
                    corner_list_1 = lattice_array[column_idx][previous_row_idx][0]
                    corner_list_2 = lattice_array[column_idx+1][row_idx][0]
                    corner_list_1, corner_list_2 = merge_cell_corners(corner_list_1, corner_list_2, "y", True)
                    lattice_array[column_idx][previous_row_idx][0] = corner_list_1
                    lattice_array[column_idx+1][row_idx][0] = corner_list_2
                    row_idx += 1                

    # Merge upwards
    for column_idx in range(len(lattice_array)):
        for row_idx in range(len(lattice_array[column_idx]) - 1):
            corner_list_1 = lattice_array[column_idx][row_idx][0]
            corner_list_2 = lattice_array[column_idx][row_idx+1][0]
            # print(corner_list_1, corner_list_2)
            corner_list_1, corner_list_2 = merge_cell_corners(corner_list_1, corner_list_2, "z", False)
            lattice_array[column_idx][row_idx][0] = corner_list_1
            lattice_array[column_idx][row_idx+1][0] = corner_list_2
            # print(corner_list_1, corner_list_2)

    for column_idx in range(len(lattice_array)):
        strut_thickness = cell_size*0.01*(40-cell_size_arr[column_idx])
        
        for cell in lattice_array[column_idx]:
            if len(cell) == 3:
                corner_list, central, face_centre_list = cell
                central[0] = np.sum(corner_list, axis=0)/8
                for face_idx in range(4):
                    face_centre = np.zeros(3)
                    for corner_idx in face_centre_components_list[face_idx]:
                        face_centre += corner_list[corner_idx]
                    face_centre_list[face_idx] = face_centre/4
                fcc_cell(corner_list, central, face_centre_list, strut_thickness)
            else:
                corner_list, central = cell
                central[0] = np.sum(corner_list, axis=0)/8
                bcc_cell(corner_list, central, strut_thickness)         
        # strut_thickness += 0.2
    
# face_centre_list_origin = np.array([(0,0.5,0.5), (0.5,1,0.5), (1,0.5,0.5), (0.5,0,0.5), (0.5,0.5,1), (0.5,0.5,0)]) 

In [92]:
myv.figure(bgcolor=(1,1,1))
cell_size_arr = [10]
# cell_size_arr = [20,16,13,10,7,5]
# cell_size_arr = range(30,5,-2)
cell_size_arr = list(range(5,30,2)) + list(range(30,5,-2))  
# cell_size_arr = [20,15,10,5]
# cell_size_arr = [13,16]
cell_size_arr = [4,6,8,14,20,24,20,14,16,20,16,12,8,6,4]

create_gradient_lattice(cell_size_arr, 120, 20)
myv.show()